<a href="https://colab.research.google.com/github/mtzig/LIDC_GDRO/blob/main/civil_comments_script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
!nvidia-smi -L  

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-6bb9d08a-3739-a3af-4ab9-607a204e9175)


In [3]:
!git clone https://github.com/MsAsterisk1/ERM_GDRO.git
%cd /content/ERM_GDRO

fatal: destination path 'ERM_GDRO' already exists and is not an empty directory.
/content/ERM_GDRO


In [4]:
!git pull

Already up to date.


In [5]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
import pandas as pd
from utils.process_data_utils import  get_dataloaders, get_CivilComments_Datasets, get_CivilComments_df, split_dataset, get_partitioned_dataloader
from loss import ERMLoss, GDROLoss, ERMGDROLoss, UpweightLoss
from train_eval import train, evaluate, train_epochs
import torch
from models import BertClassifier
from math import ceil
import transformers

Now lets try getting Bert working

In [7]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
DEVICE

'cuda'

In [8]:
df = get_CivilComments_df()

In [ ]:
datasets = get_CivilComments_Datasets(CC_df=df, device=DEVICE)

In [ ]:
tr,cv,test = get_dataloaders(datasets, (16,32,32), split=True)

This is how we would initialize model

In [ ]:
model = BertClassifier(device=DEVICE)
epochs = 5
# loss_fn = ERMLoss(model, torch.nn.CrossEntropyLoss())
loss_fn = ERMGDROLoss(model, torch.nn.CrossEntropyLoss(), 0.01, 2, 0.5, partitioned=True)

num_training_steps = ceil(tr.batches_per_epoch() / 16) * epochs
optimizer = torch.optim.AdamW(model.parameters(), lr=0.00001, weight_decay=0.01)

scheduler = transformers.get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)
#note scheduler is in wrong place in train_eval currently

train_epochs(epochs, tr, cv, test, model, loss_fn, optimizer, scheduler=scheduler, vector_subclass=True, verbose=True, num_subclasses=18, save_weights_name='CC_ERM_GDRO', gradient_clip = 1)

In [ ]:
_ = evaluate(test, model, 18, vector_subclass=True, replacement=False, get_loss=False, verbose=True)